In [1]:
import sys
import os

PROJECT_DIR = '/home/evgenshuben/Desktop/gitReps/YandexCup/'
sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)

In [27]:
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose, initialize_config_dir
from hydra.utils import instantiate

import torch.nn as nn
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
import faiss
from sklearn.model_selection import train_test_split

In [3]:
%load_ext autoreload
%autoreload 2

from scr.train import Trainer
from scr.data.dataset import CoverDatasetClassifier

# Load data

In [4]:
nearest_track_idx_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/reranker/train_dataset_pd.pq')

In [28]:
nearest_track_idx_pd.head()

,track_id,neighbors,negatives,positives
0,343223,"[343223, 314991, 254137, 200291, 101812, 12627...","[314991, 254137, 200291, 101812, 126276, 35797...","[307473, 361210, 101132, 343223]"
1,361210,"[361210, 101132, 307473, 343223, 281680, 33874...","[281680, 338741, 204228, 182103, 189128, 23351...","[59234, 101132, 307473, 343223, 361210]"
2,114472,"[114472, 247079, 109255, 216454, 192349, 13474...","[109255, 192349, 258956, 236469, 221151, 22684...","[59234, 216454, 247079, 114472, 215467, 307473..."
3,134744,"[134744, 215467, 258956, 247079, 179511, 30816...","[258956, 179511, 308169, 146032, 136942, 29370...","[59234, 216454, 247079, 114472, 215467, 101132..."
4,271362,"[271362, 315298, 144962, 332689, 180183, 19886...","[315298, 144962, 332689, 180183, 198867, 29852...",[271362]


In [11]:
nearest_track_idx_pd[nearest_track_idx_pd['positives'].str.len() == 0]

,track_id,neighbors,negatives,positives


In [30]:
def split_train_val(df, test_size=0.2, random_state=42):  
    unique_track_ids = df['track_id'].unique()
    train_ids, val_ids = train_test_split(unique_track_ids, test_size=test_size, random_state=random_state)
    
    train_df = df[df['track_id'].isin(train_ids)].reset_index(drop=True)
    val_df = df[df['track_id'].isin(val_ids)].reset_index(drop=True)
    return train_df, val_df

In [31]:
def prepare_reranker_dataset(df, negative_sample_size=None, random_state=42):
    np.random.seed(random_state)
    

    flat_data = []

    for _, row in df.iterrows():
        track_id = row['track_id']
        
        for pos_id in row['positives']:
            flat_data.append({'track_id': track_id, 'candidate_id': pos_id, 'target': 1})
        
        if negative_sample_size is not None:
            sampled_negatives = np.random.choice(row['negatives'], size=min(negative_sample_size, len(row['negatives'])), replace=False)
        else:
            sampled_negatives = row['negatives']
        
        for neg_id in sampled_negatives:
            flat_data.append({'track_id': track_id, 'candidate_id': neg_id, 'target': 0})

    flat_df = pd.DataFrame(flat_data)

    return flat_df

Нет вообще треков, у которых среди топ-200 не нашлось бы позитивов

In [44]:
df_sampled = nearest_track_idx_pd.sample(n=50000, random_state=42)
df_sampled.shape

(50000, 4)

In [45]:
train_df, val_df = split_train_val(df_sampled, test_size=0.1)

In [46]:
train_flat_pd = prepare_reranker_dataset(train_df, negative_sample_size=30)
val_flat_pd = prepare_reranker_dataset(val_df, negative_sample_size=None)

print('train shape:', train_flat_pd.shape)
print('val shape:', val_flat_pd.shape)

train shape: (2346707, 3)
val shape: (1000000, 3)


###### inferense model

теперь осталось проставлять предикты от модели к датасетам

Для этого нужно загрузить модель и заинференсить ее на парах, которые получились



In [47]:
train_flat_pd

,track_id,candidate_id,target
0,108293,108293,1
1,108293,317206,1
2,108293,319104,0
3,108293,289551,0
4,108293,263026,0
...,...,...,...
2346702,310815,272467,0
2346703,310815,283433,0
2346704,310815,325370,0
2346705,310815,347644,0
